In [0]:
import pandas as pd
import numpy as np
import requests
import re

In [0]:
df = pd.read_json('https://greendeck-datasets-2.s3.amazonaws.com/netaporter_gb_similar.json', lines=True)

In [0]:
oid_col = ['_id', 'website_id']
for col in oid_col:
  temp = pd.DataFrame(list(df[col]))
  df[col] = temp['$oid']

df.head()

,_id,name,sku,url,brand,media,meta,website_id,price,description_text,spider,stock,classification,created_at,updated_at,similar_products,positioning,lv_url,price_changes,price_positioning,price_positioning_text,sizes
0,5da822166504fb65da005a59,Prada Feather-embellished silk-twill robe,1000954,https://www.net-a-porter.com/gb/en/product/100...,"{'name': 'prada', 'sub_brand': ''}","{'standard': [{'order': 1, 'url': 'http://cach...","{'breadcrumbs': {'1': 'Clothing ', '2': ' Jack...",5da6d40110309200045874e6,"{'offer_price': {'currency': 'GBP', 'value': 1...","Gray and black silk-twill, black feathers Slip...",netaporter_gb_pv,{'available': False},"{'l1': 'clothing', 'l2': 'nightwear', 'l3': ''...",{'$date': '2019-10-21T00:00:00.000Z'},{'$date': '2020-01-18T06:04:52.517Z'},"{'meta': {'total_results': 1, 'min_price': {'r...","{'page': 236, 'rank': 20}",,[{'date': {'$date': '2019-10-21T00:00:00.000Z'...,0.0,expensive,NaN
1,5da822607d3a9baa211f85f4,Prada Hooded rubber-appliquéd color-block gab...,1000960,https://www.net-a-porter.com/gb/en/product/100...,"{'name': 'prada', 'sub_brand': ''}","{'standard': [{'order': 1, 'url': 'http://cach...","{'breadcrumbs': {'1': 'Clothing ', '2': ' Jack...",5da6d40110309200045874e6,"{'offer_price': {'currency': 'GBP', 'value': 1...","Black, white and gray gabardine Concealed zip ...",netaporter_gb_pv,{'available': True},"{'l1': 'clothing', 'l2': 'outerwear', 'l3': 'c...",{'$date': '2019-10-21T00:00:00.000Z'},{'$date': '2020-01-18T06:07:47.995Z'},"{'meta': {'total_results': 1, 'min_price': {'r...","{'page': 146, 'rank': 14}",,[{'date': {'$date': '2019-10-21T00:00:00.000Z'...,100.0,cheaper,NaN
2,5da8221a6504fb65da005a7b,Prada Printed silk crepe de chine flared pant...,1000995,https://www.net-a-porter.com/gb/en/product/100...,"{'name': 'prada', 'sub_brand': ''}","{'standard': [{'order': 1, 'url': 'http://cach...","{'breadcrumbs': {'1': 'Clothing ', '2': ' Pant...",5da6d40110309200045874e6,"{'offer_price': {'currency': 'GBP', 'value': 8...","Black, red and cream silk crepe de chine Conce...",netaporter_gb_pv,{'available': True},"{'l1': 'clothing', 'l2': 'bottomwear', 'l3': '...",{'$date': '2019-10-21T00:00:00.000Z'},{'$date': '2020-01-18T06:07:49.170Z'},"{'meta': {'total_results': 1, 'min_price': {'r...","{'page': 133, 'rank': 35}",,[{'date': {'$date': '2019-10-21T00:00:00.000Z'...,100.0,cheaper,NaN
3,5da8218b6504fb65da004fb2,Prada Silk-twill shorts,1000996,https://www.net-a-porter.com/gb/en/product/100...,"{'name': 'prada', 'sub_brand': ''}","{'standard': [{'order': 1, 'url': 'http://cach...","{'breadcrumbs': {'1': 'Clothing ', '2': ' Shor...",5da6d40110309200045874e6,"{'offer_price': {'currency': 'GBP', 'value': 7...",Pastel-pink and black silk-twill Button and co...,netaporter_gb_pv,{'available': True},"{'l1': 'clothing', 'l2': 'bottomwear', 'l3': '...",{'$date': '2019-10-21T00:00:00.000Z'},{'$date': '2020-01-18T06:04:54.021Z'},"{'meta': {'total_results': 1, 'min_price': {'r...","{'page': 142, 'rank': 58}",,[{'date': {'$date': '2019-10-21T00:00:00.000Z'...,100.0,cheaper,NaN
4,5da823b76504fb65cb261ac9,Prada Plissé-organza skirt,1000997,https://www.net-a-porter.com/gb/en/product/100...,"{'name': 'prada', 'sub_brand': ''}","{'standard': [{'order': 1, 'url': 'http://cach...","{'breadcrumbs': {'1': 'Clothing ', '2': ' Skir...",5da6d40110309200045874e6,"{'offer_price': {'currency': 'GBP', 'value': 8...",Pastel-pink plissé-organza Concealed zip faste...,netaporter_gb_pv,{'available': True},"{'l1': 'clothing', 'l2': 'skirts', 'l3': '', '...",{'$date': '2019-10-21T00:00:00.000Z'},{'$date': '2020-01-18T06:07:51.463Z'},"{'meta': {'total_results': 1, 'min_price': {'r...","{'page': 37, 'rank': 39}",,[{'date': {'$date': '2019-10-21T00:00:00.000Z'...,100.0,cheaper,NaN


In [0]:
# brand = list(df.brand.head())
df['brand'] = [brand['name'] for brand in list(df.brand)]
df.head()

,_id,name,sku,url,brand,media,meta,website_id,price,description_text,spider,stock,classification,created_at,updated_at,similar_products,positioning,lv_url,price_changes,price_positioning,price_positioning_text,sizes
0,5da822166504fb65da005a59,Prada Feather-embellished silk-twill robe,1000954,https://www.net-a-porter.com/gb/en/product/100...,prada,"{'standard': [{'order': 1, 'url': 'http://cach...","{'breadcrumbs': {'1': 'Clothing ', '2': ' Jack...",5da6d40110309200045874e6,"{'offer_price': {'currency': 'GBP', 'value': 1...","Gray and black silk-twill, black feathers Slip...",netaporter_gb_pv,{'available': False},"{'l1': 'clothing', 'l2': 'nightwear', 'l3': ''...",{'$date': '2019-10-21T00:00:00.000Z'},{'$date': '2020-01-18T06:04:52.517Z'},"{'meta': {'total_results': 1, 'min_price': {'r...","{'page': 236, 'rank': 20}",,[{'date': {'$date': '2019-10-21T00:00:00.000Z'...,0.0,expensive,NaN
1,5da822607d3a9baa211f85f4,Prada Hooded rubber-appliquéd color-block gab...,1000960,https://www.net-a-porter.com/gb/en/product/100...,prada,"{'standard': [{'order': 1, 'url': 'http://cach...","{'breadcrumbs': {'1': 'Clothing ', '2': ' Jack...",5da6d40110309200045874e6,"{'offer_price': {'currency': 'GBP', 'value': 1...","Black, white and gray gabardine Concealed zip ...",netaporter_gb_pv,{'available': True},"{'l1': 'clothing', 'l2': 'outerwear', 'l3': 'c...",{'$date': '2019-10-21T00:00:00.000Z'},{'$date': '2020-01-18T06:07:47.995Z'},"{'meta': {'total_results': 1, 'min_price': {'r...","{'page': 146, 'rank': 14}",,[{'date': {'$date': '2019-10-21T00:00:00.000Z'...,100.0,cheaper,NaN
2,5da8221a6504fb65da005a7b,Prada Printed silk crepe de chine flared pant...,1000995,https://www.net-a-porter.com/gb/en/product/100...,prada,"{'standard': [{'order': 1, 'url': 'http://cach...","{'breadcrumbs': {'1': 'Clothing ', '2': ' Pant...",5da6d40110309200045874e6,"{'offer_price': {'currency': 'GBP', 'value': 8...","Black, red and cream silk crepe de chine Conce...",netaporter_gb_pv,{'available': True},"{'l1': 'clothing', 'l2': 'bottomwear', 'l3': '...",{'$date': '2019-10-21T00:00:00.000Z'},{'$date': '2020-01-18T06:07:49.170Z'},"{'meta': {'total_results': 1, 'min_price': {'r...","{'page': 133, 'rank': 35}",,[{'date': {'$date': '2019-10-21T00:00:00.000Z'...,100.0,cheaper,NaN
3,5da8218b6504fb65da004fb2,Prada Silk-twill shorts,1000996,https://www.net-a-porter.com/gb/en/product/100...,prada,"{'standard': [{'order': 1, 'url': 'http://cach...","{'breadcrumbs': {'1': 'Clothing ', '2': ' Shor...",5da6d40110309200045874e6,"{'offer_price': {'currency': 'GBP', 'value': 7...",Pastel-pink and black silk-twill Button and co...,netaporter_gb_pv,{'available': True},"{'l1': 'clothing', 'l2': 'bottomwear', 'l3': '...",{'$date': '2019-10-21T00:00:00.000Z'},{'$date': '2020-01-18T06:04:54.021Z'},"{'meta': {'total_results': 1, 'min_price': {'r...","{'page': 142, 'rank': 58}",,[{'date': {'$date': '2019-10-21T00:00:00.000Z'...,100.0,cheaper,NaN
4,5da823b76504fb65cb261ac9,Prada Plissé-organza skirt,1000997,https://www.net-a-porter.com/gb/en/product/100...,prada,"{'standard': [{'order': 1, 'url': 'http://cach...","{'breadcrumbs': {'1': 'Clothing ', '2': ' Skir...",5da6d40110309200045874e6,"{'offer_price': {'currency': 'GBP', 'value': 8...",Pastel-pink plissé-organza Concealed zip faste...,netaporter_gb_pv,{'available': True},"{'l1': 'clothing', 'l2': 'skirts', 'l3': '', '...",{'$date': '2019-10-21T00:00:00.000Z'},{'$date': '2020-01-18T06:07:51.463Z'},"{'meta': {'total_results': 1, 'min_price': {'r...","{'page': 37, 'rank': 39}",,[{'date': {'$date': '2019-10-21T00:00:00.000Z'...,100.0,cheaper,NaN


In [0]:
# df.media.head()

In [0]:
# df.price.head(10)

0    {'offer_price': {'currency': 'GBP', 'value': 1...
1    {'offer_price': {'currency': 'GBP', 'value': 1...
2    {'offer_price': {'currency': 'GBP', 'value': 8...
3    {'offer_price': {'currency': 'GBP', 'value': 7...
4    {'offer_price': {'currency': 'GBP', 'value': 8...
5    {'offer_price': {'currency': 'GBP', 'value': 1...
6    {'offer_price': {'currency': 'GBP', 'value': 1...
7    {'offer_price': {'currency': 'GBP', 'value': 5...
8    {'offer_price': {'currency': 'GBP', 'value': 4...
9    {'offer_price': {'currency': 'GBP', 'value': 1...
Name: price, dtype: object

## Question 1

In [0]:
price_df = pd.DataFrame(list(df['price']))
price_df.head()

,offer_price,regular_price,basket_price
0,"{'currency': 'GBP', 'value': 1160.0}","{'currency': 'GBP', 'value': 1160.0}","{'value': 1160.0, 'currency': 'GBP'}"
1,"{'currency': 'GBP', 'value': 1210.0}","{'currency': 'GBP', 'value': 1210.0}","{'value': 1210.0, 'currency': 'GBP'}"
2,"{'currency': 'GBP', 'value': 855.0}","{'currency': 'GBP', 'value': 855.0}","{'value': 855.0, 'currency': 'GBP'}"
3,"{'currency': 'GBP', 'value': 700.0}","{'currency': 'GBP', 'value': 700.0}","{'value': 700.0, 'currency': 'GBP'}"
4,"{'currency': 'GBP', 'value': 880.0}","{'currency': 'GBP', 'value': 880.0}","{'value': 880.0, 'currency': 'GBP'}"


In [0]:
# df[df[['SELECTIO']].applymap(np.isreal).all(1)]
# for cols in price_df.columns:
#   price_df[cols] = price_df[cols].str.extract('(\d*\.\d+|\d+)', expand=False).astype(float)
# price_df.head()

,offer_price,regular_price,basket_price
0,NaN,NaN,NaN
1,NaN,NaN,NaN
2,NaN,NaN,NaN
3,NaN,NaN,NaN
4,NaN,NaN,NaN


In [0]:
# price_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4904 entries, 0 to 4903
Data columns (total 3 columns):
offer_price      4904 non-null object
regular_price    4904 non-null object
basket_price     4904 non-null object
dtypes: object(3)
memory usage: 115.1+ KB


In [0]:
# price_df['offer_price'].head()

0    {'currency': 'GBP', 'value': 1160.0}
1    {'currency': 'GBP', 'value': 1210.0}
2     {'currency': 'GBP', 'value': 855.0}
3     {'currency': 'GBP', 'value': 700.0}
4     {'currency': 'GBP', 'value': 880.0}
Name: offer_price, dtype: object

In [0]:
offer_list = []
regular_list = []
basket_list = []
for offer, regular, basket in zip(price_df['offer_price'], price_df['regular_price'], price_df['basket_price']):
  offer_list.append(offer['value'])
  regular_list.append(regular['value'])
  basket_list.append(basket['value'])
price_df['offer_price'] = pd.Series(offer_list)
price_df['regular_price'] = pd.Series(regular_list)
price_df['basket_price'] = pd.Series(basket_list)
price_df.head()

,offer_price,regular_price,basket_price
0,1160.0,1160.0,1160.0
1,1210.0,1210.0,1210.0
2,855.0,855.0,855.0
3,700.0,700.0,700.0
4,880.0,880.0,880.0


In [0]:
df = pd.concat([df, price_df], axis=1)
df.head()

,_id,name,sku,url,brand,media,meta,website_id,price,description_text,spider,stock,classification,created_at,updated_at,similar_products,positioning,lv_url,price_changes,price_positioning,price_positioning_text,sizes,offer_price,regular_price,basket_price
0,5da822166504fb65da005a59,Prada Feather-embellished silk-twill robe,1000954,https://www.net-a-porter.com/gb/en/product/100...,prada,"{'standard': [{'order': 1, 'url': 'http://cach...","{'breadcrumbs': {'1': 'Clothing ', '2': ' Jack...",5da6d40110309200045874e6,"{'offer_price': {'currency': 'GBP', 'value': 1...","Gray and black silk-twill, black feathers Slip...",netaporter_gb_pv,{'available': False},"{'l1': 'clothing', 'l2': 'nightwear', 'l3': ''...",{'$date': '2019-10-21T00:00:00.000Z'},{'$date': '2020-01-18T06:04:52.517Z'},"{'meta': {'total_results': 1, 'min_price': {'r...","{'page': 236, 'rank': 20}",,[{'date': {'$date': '2019-10-21T00:00:00.000Z'...,0.0,expensive,NaN,1160.0,1160.0,1160.0
1,5da822607d3a9baa211f85f4,Prada Hooded rubber-appliquéd color-block gab...,1000960,https://www.net-a-porter.com/gb/en/product/100...,prada,"{'standard': [{'order': 1, 'url': 'http://cach...","{'breadcrumbs': {'1': 'Clothing ', '2': ' Jack...",5da6d40110309200045874e6,"{'offer_price': {'currency': 'GBP', 'value': 1...","Black, white and gray gabardine Concealed zip ...",netaporter_gb_pv,{'available': True},"{'l1': 'clothing', 'l2': 'outerwear', 'l3': 'c...",{'$date': '2019-10-21T00:00:00.000Z'},{'$date': '2020-01-18T06:07:47.995Z'},"{'meta': {'total_results': 1, 'min_price': {'r...","{'page': 146, 'rank': 14}",,[{'date': {'$date': '2019-10-21T00:00:00.000Z'...,100.0,cheaper,NaN,1210.0,1210.0,1210.0
2,5da8221a6504fb65da005a7b,Prada Printed silk crepe de chine flared pant...,1000995,https://www.net-a-porter.com/gb/en/product/100...,prada,"{'standard': [{'order': 1, 'url': 'http://cach...","{'breadcrumbs': {'1': 'Clothing ', '2': ' Pant...",5da6d40110309200045874e6,"{'offer_price': {'currency': 'GBP', 'value': 8...","Black, red and cream silk crepe de chine Conce...",netaporter_gb_pv,{'available': True},"{'l1': 'clothing', 'l2': 'bottomwear', 'l3': '...",{'$date': '2019-10-21T00:00:00.000Z'},{'$date': '2020-01-18T06:07:49.170Z'},"{'meta': {'total_results': 1, 'min_price': {'r...","{'page': 133, 'rank': 35}",,[{'date': {'$date': '2019-10-21T00:00:00.000Z'...,100.0,cheaper,NaN,855.0,855.0,855.0
3,5da8218b6504fb65da004fb2,Prada Silk-twill shorts,1000996,https://www.net-a-porter.com/gb/en/product/100...,prada,"{'standard': [{'order': 1, 'url': 'http://cach...","{'breadcrumbs': {'1': 'Clothing ', '2': ' Shor...",5da6d40110309200045874e6,"{'offer_price': {'currency': 'GBP', 'value': 7...",Pastel-pink and black silk-twill Button and co...,netaporter_gb_pv,{'available': True},"{'l1': 'clothing', 'l2': 'bottomwear', 'l3': '...",{'$date': '2019-10-21T00:00:00.000Z'},{'$date': '2020-01-18T06:04:54.021Z'},"{'meta': {'total_results': 1, 'min_price': {'r...","{'page': 142, 'rank': 58}",,[{'date': {'$date': '2019-10-21T00:00:00.000Z'...,100.0,cheaper,NaN,700.0,700.0,700.0
4,5da823b76504fb65cb261ac9,Prada Plissé-organza skirt,1000997,https://www.net-a-porter.com/gb/en/product/100...,prada,"{'standard': [{'order': 1, 'url': 'http://cach...","{'breadcrumbs': {'1': 'Clothing ', '2': ' Skir...",5da6d40110309200045874e6,"{'offer_price': {'currency': 'GBP', 'value': 8...",Pastel-pink plissé-organza Concealed zip faste...,netaporter_gb_pv,{'available': True},"{'l1': 'clothing', 'l2': 'skirts', 'l3': '', '...",{'$date': '2019-10-21T00:00:00.000Z'},{'$date': '2020-01-18T06:07:51.463Z'},"{'meta': {'total_results': 1, 'min_price': {'r...","{'page': 37, 'rank': 39}",,[{'date': {'$date': '2019-10-21T00:00:00.000Z'...,100.0,cheaper,NaN,880.0,880.0,880.0


In [0]:
# price_df.info()

In [0]:
# price_df['id'] = df['_id']
# price_df.set_index('id', inplace=True)

In [0]:
# price_df.head(5)

,offer_price,regular_price,basket_price,id
0,1160.0,1160.0,1160.0,5da822166504fb65da005a59
1,1210.0,1210.0,1210.0,5da822607d3a9baa211f85f4
2,855.0,855.0,855.0,5da8221a6504fb65da005a7b
3,700.0,700.0,700.0,5da8218b6504fb65da004fb2
4,880.0,880.0,880.0,5da823b76504fb65cb261ac9


In [0]:
df['discount_price'] = df['regular_price'] - df['offer_price']
df.head()

,_id,name,sku,url,brand,media,meta,website_id,price,description_text,spider,stock,classification,created_at,updated_at,similar_products,positioning,lv_url,price_changes,price_positioning,price_positioning_text,sizes,offer_price,regular_price,basket_price,discount_price
0,5da822166504fb65da005a59,Prada Feather-embellished silk-twill robe,1000954,https://www.net-a-porter.com/gb/en/product/100...,prada,"{'standard': [{'order': 1, 'url': 'http://cach...","{'breadcrumbs': {'1': 'Clothing ', '2': ' Jack...",5da6d40110309200045874e6,"{'offer_price': {'currency': 'GBP', 'value': 1...","Gray and black silk-twill, black feathers Slip...",netaporter_gb_pv,{'available': False},"{'l1': 'clothing', 'l2': 'nightwear', 'l3': ''...",{'$date': '2019-10-21T00:00:00.000Z'},{'$date': '2020-01-18T06:04:52.517Z'},"{'meta': {'total_results': 1, 'min_price': {'r...","{'page': 236, 'rank': 20}",,[{'date': {'$date': '2019-10-21T00:00:00.000Z'...,0.0,expensive,NaN,1160.0,1160.0,1160.0,0.0
1,5da822607d3a9baa211f85f4,Prada Hooded rubber-appliquéd color-block gab...,1000960,https://www.net-a-porter.com/gb/en/product/100...,prada,"{'standard': [{'order': 1, 'url': 'http://cach...","{'breadcrumbs': {'1': 'Clothing ', '2': ' Jack...",5da6d40110309200045874e6,"{'offer_price': {'currency': 'GBP', 'value': 1...","Black, white and gray gabardine Concealed zip ...",netaporter_gb_pv,{'available': True},"{'l1': 'clothing', 'l2': 'outerwear', 'l3': 'c...",{'$date': '2019-10-21T00:00:00.000Z'},{'$date': '2020-01-18T06:07:47.995Z'},"{'meta': {'total_results': 1, 'min_price': {'r...","{'page': 146, 'rank': 14}",,[{'date': {'$date': '2019-10-21T00:00:00.000Z'...,100.0,cheaper,NaN,1210.0,1210.0,1210.0,0.0
2,5da8221a6504fb65da005a7b,Prada Printed silk crepe de chine flared pant...,1000995,https://www.net-a-porter.com/gb/en/product/100...,prada,"{'standard': [{'order': 1, 'url': 'http://cach...","{'breadcrumbs': {'1': 'Clothing ', '2': ' Pant...",5da6d40110309200045874e6,"{'offer_price': {'currency': 'GBP', 'value': 8...","Black, red and cream silk crepe de chine Conce...",netaporter_gb_pv,{'available': True},"{'l1': 'clothing', 'l2': 'bottomwear', 'l3': '...",{'$date': '2019-10-21T00:00:00.000Z'},{'$date': '2020-01-18T06:07:49.170Z'},"{'meta': {'total_results': 1, 'min_price': {'r...","{'page': 133, 'rank': 35}",,[{'date': {'$date': '2019-10-21T00:00:00.000Z'...,100.0,cheaper,NaN,855.0,855.0,855.0,0.0
3,5da8218b6504fb65da004fb2,Prada Silk-twill shorts,1000996,https://www.net-a-porter.com/gb/en/product/100...,prada,"{'standard': [{'order': 1, 'url': 'http://cach...","{'breadcrumbs': {'1': 'Clothing ', '2': ' Shor...",5da6d40110309200045874e6,"{'offer_price': {'currency': 'GBP', 'value': 7...",Pastel-pink and black silk-twill Button and co...,netaporter_gb_pv,{'available': True},"{'l1': 'clothing', 'l2': 'bottomwear', 'l3': '...",{'$date': '2019-10-21T00:00:00.000Z'},{'$date': '2020-01-18T06:04:54.021Z'},"{'meta': {'total_results': 1, 'min_price': {'r...","{'page': 142, 'rank': 58}",,[{'date': {'$date': '2019-10-21T00:00:00.000Z'...,100.0,cheaper,NaN,700.0,700.0,700.0,0.0
4,5da823b76504fb65cb261ac9,Prada Plissé-organza skirt,1000997,https://www.net-a-porter.com/gb/en/product/100...,prada,"{'standard': [{'order': 1, 'url': 'http://cach...","{'breadcrumbs': {'1': 'Clothing ', '2': ' Skir...",5da6d40110309200045874e6,"{'offer_price': {'currency': 'GBP', 'value': 8...",Pastel-pink plissé-organza Concealed zip faste...,netaporter_gb_pv,{'available': True},"{'l1': 'clothing', 'l2': 'skirts', 'l3': '', '...",{'$date': '2019-10-21T00:00:00.000Z'},{'$date': '2020-01-18T06:07:51.463Z'},"{'meta': {'total_results': 1, 'min_price': {'r...","{'page': 37, 'rank': 39}",,[{'date': {'$date': '2019-10-21T00:00:00.000Z'...,100.0,cheaper,NaN,880.0,880.0,880.0,0.0


## Question 2

In [0]:
# list(df.similar_products)[:1]
# for item in df.similar_products:
#   print(item['meta']['min_price'])
#   print(item['website_results'].keys())
#   break

In [0]:
# df.brand.tail(10)

4894    gucci
4895    gucci
4896    gucci
4897    gucci
4898    gucci
4899    gucci
4900    gucci
4901    gucci
4902    gucci
4903    gucci
Name: brand, dtype: object

In [0]:
# price_df.reset_index(inplace=True)
# price_df['brand'] = df['brand']
# price_df.head()

,offer_price,regular_price,basket_price,id,discount_price,brand
0,1160.0,1160.0,1160.0,5da822166504fb65da005a59,0.0,prada
1,1210.0,1210.0,1210.0,5da822607d3a9baa211f85f4,0.0,prada
2,855.0,855.0,855.0,5da8221a6504fb65da005a7b,0.0,prada
3,700.0,700.0,700.0,5da8218b6504fb65da004fb2,0.0,prada
4,880.0,880.0,880.0,5da823b76504fb65cb261ac9,0.0,prada


In [0]:
discount_by_brand = df.groupby('brand').mean()
discount_by_brand.head()

,sku,price_positioning,offer_price,regular_price,basket_price,discount_price
brand,,,,,,
16arlington,1.163155e+06,0.000000,441.000000,630.000000,441.000000,189.000000
3.1 phillip lim,1.155970e+06,0.000000,327.388889,423.444444,327.944444,96.055556
a.p.c. atelier de production et de création,1.006246e+06,100.000000,155.000000,155.000000,160.000000,0.000000
a.w.a.k.e. mode,1.173544e+06,0.000000,280.142857,396.285714,281.571429,116.142857
acne studios,1.135948e+06,10.851279,326.477941,467.441176,328.757353,140.963235


In [0]:
discount_by_brand.info()

<class 'pandas.core.frame.DataFrame'>
Index: 359 entries, 16arlington to zimmermann
Data columns (total 4 columns):
offer_price       359 non-null float64
regular_price     359 non-null float64
basket_price      359 non-null float64
discount_price    359 non-null float64
dtypes: float64(4)
memory usage: 24.0+ KB


In [0]:
  # price_df.brand.value_counts()

gucci                300
balenciaga           267
prada                253
chloé                164
saint laurent        126
                    ... 
eres                   1
la mer                 1
wolford                1
jade swim              1
vivienne westwood      1
Name: brand, Length: 359, dtype: int64

In [0]:
# df.columns

In [0]:
# df.website_id.value_counts()

5da6d40110309200045874e6    4904
Name: website_id, dtype: int64

## Question 3
### NAP products where they are selling at a price higher than any of the competition.

In [0]:
expensive_list=[]
price_list=[]
for item, basket in zip(df.similar_products, df.basket_price):
  # print(item, basket)
  temp_list = []
  temp_price=[]
  # print(item['meta'])
  comp_id = list(item['website_results'].keys())
  for val in item['website_results'].keys():
    if item['website_results'][val]['meta']['total_results'] == 1:
      # print('Yes')
      temp = item['website_results'][val]['meta']['min_price']['basket']
      if basket > temp:
        temp_list.append(val)
        temp_price.append(temp)
    # print(item['website_results'][val]['meta']['max_price'])
  expensive_list.append(temp_list)
  price_list.append(temp_price)

In [0]:
df['expensive_id'] = pd.Series(expensive_list)
df['comp_basket_price'] = pd.Series(price_list)
df.head()

,_id,name,sku,url,brand,media,meta,website_id,price,description_text,spider,stock,classification,created_at,updated_at,similar_products,positioning,lv_url,price_changes,price_positioning,price_positioning_text,sizes,offer_price,regular_price,basket_price,discount_price,expensive_id,comp_basket_price
0,5da822166504fb65da005a59,Prada Feather-embellished silk-twill robe,1000954,https://www.net-a-porter.com/gb/en/product/100...,prada,"{'standard': [{'order': 1, 'url': 'http://cach...","{'breadcrumbs': {'1': 'Clothing ', '2': ' Jack...",5da6d40110309200045874e6,"{'offer_price': {'currency': 'GBP', 'value': 1...","Gray and black silk-twill, black feathers Slip...",netaporter_gb_pv,{'available': False},"{'l1': 'clothing', 'l2': 'nightwear', 'l3': ''...",{'$date': '2019-10-21T00:00:00.000Z'},{'$date': '2020-01-18T06:04:52.517Z'},"{'meta': {'total_results': 1, 'min_price': {'r...","{'page': 236, 'rank': 20}",,[{'date': {'$date': '2019-10-21T00:00:00.000Z'...,0.0,expensive,NaN,1160.0,1160.0,1160.0,0.0,[],[]
1,5da822607d3a9baa211f85f4,Prada Hooded rubber-appliquéd color-block gab...,1000960,https://www.net-a-porter.com/gb/en/product/100...,prada,"{'standard': [{'order': 1, 'url': 'http://cach...","{'breadcrumbs': {'1': 'Clothing ', '2': ' Jack...",5da6d40110309200045874e6,"{'offer_price': {'currency': 'GBP', 'value': 1...","Black, white and gray gabardine Concealed zip ...",netaporter_gb_pv,{'available': True},"{'l1': 'clothing', 'l2': 'outerwear', 'l3': 'c...",{'$date': '2019-10-21T00:00:00.000Z'},{'$date': '2020-01-18T06:07:47.995Z'},"{'meta': {'total_results': 1, 'min_price': {'r...","{'page': 146, 'rank': 14}",,[{'date': {'$date': '2019-10-21T00:00:00.000Z'...,100.0,cheaper,NaN,1210.0,1210.0,1210.0,0.0,[5da94e940ffeca000172b12a],[605.0]
2,5da8221a6504fb65da005a7b,Prada Printed silk crepe de chine flared pant...,1000995,https://www.net-a-porter.com/gb/en/product/100...,prada,"{'standard': [{'order': 1, 'url': 'http://cach...","{'breadcrumbs': {'1': 'Clothing ', '2': ' Pant...",5da6d40110309200045874e6,"{'offer_price': {'currency': 'GBP', 'value': 8...","Black, red and cream silk crepe de chine Conce...",netaporter_gb_pv,{'available': True},"{'l1': 'clothing', 'l2': 'bottomwear', 'l3': '...",{'$date': '2019-10-21T00:00:00.000Z'},{'$date': '2020-01-18T06:07:49.170Z'},"{'meta': {'total_results': 1, 'min_price': {'r...","{'page': 133, 'rank': 35}",,[{'date': {'$date': '2019-10-21T00:00:00.000Z'...,100.0,cheaper,NaN,855.0,855.0,855.0,0.0,[5d0cc7b68a66a100014acdb0],[518.0]
3,5da8218b6504fb65da004fb2,Prada Silk-twill shorts,1000996,https://www.net-a-porter.com/gb/en/product/100...,prada,"{'standard': [{'order': 1, 'url': 'http://cach...","{'breadcrumbs': {'1': 'Clothing ', '2': ' Shor...",5da6d40110309200045874e6,"{'offer_price': {'currency': 'GBP', 'value': 7...",Pastel-pink and black silk-twill Button and co...,netaporter_gb_pv,{'available': True},"{'l1': 'clothing', 'l2': 'bottomwear', 'l3': '...",{'$date': '2019-10-21T00:00:00.000Z'},{'$date': '2020-01-18T06:04:54.021Z'},"{'meta': {'total_results': 1, 'min_price': {'r...","{'page': 142, 'rank': 58}",,[{'date': {'$date': '2019-10-21T00:00:00.000Z'...,100.0,cheaper,NaN,700.0,700.0,700.0,0.0,[5da94e940ffeca000172b12a],[357.0]
4,5da823b76504fb65cb261ac9,Prada Plissé-organza skirt,1000997,https://www.net-a-porter.com/gb/en/product/100...,prada,"{'standard': [{'order': 1, 'url': 'http://cach...","{'breadcrumbs': {'1': 'Clothing ', '2': ' Skir...",5da6d40110309200045874e6,"{'offer_price': {'currency': 'GBP', 'value': 8...",Pastel-pink plissé-organza Concealed zip faste...,netaporter_gb_pv,{'available': True},"{'l1': 'clothing', 'l2': 'skirts', 'l3': '', '...",{'$date': '2019-10-21T00:00:00.000Z'},{'$date': '2020-01-18T06:07:51.463Z'},"{'meta': {'total_results': 1, 'min_price': {'r...","{'page': 37, 'rank': 39}",,[{'date': {'$date': '2019-10-21T00:00:00.000Z'...,100.0,cheaper,NaN,880.0,880.0,880.0,0.0,[5da94e940ffeca000172b12a],[440.0]


In [0]:
# final_list = []
# for item, basket in zip(df.similar_products, price_df.basket_price):
#   temp_list = []
#   # print(item['website_results'].values())
#   for val in item['website_results'].keys():
#     if item['website_results'][val]['meta']['total_results'] == 1:
#       if basket > item['website_results'][val]['meta']['min_price']['basket']:
#         temp_list.append(val)
#   print(temp_list)
#   final_list.append(temp_list)
# # print(final_list)

In [0]:
df.head()

,_id,name,sku,url,brand,media,meta,website_id,price,description_text,spider,stock,classification,created_at,updated_at,similar_products,positioning,lv_url,price_changes,price_positioning,price_positioning_text,sizes,offer_price,regular_price,basket_price,discount_price,expensive_id,comp_basket_price
0,5da822166504fb65da005a59,Prada Feather-embellished silk-twill robe,1000954,https://www.net-a-porter.com/gb/en/product/100...,prada,"{'standard': [{'order': 1, 'url': 'http://cach...","{'breadcrumbs': {'1': 'Clothing ', '2': ' Jack...",5da6d40110309200045874e6,"{'offer_price': {'currency': 'GBP', 'value': 1...","Gray and black silk-twill, black feathers Slip...",netaporter_gb_pv,{'available': False},"{'l1': 'clothing', 'l2': 'nightwear', 'l3': ''...",{'$date': '2019-10-21T00:00:00.000Z'},{'$date': '2020-01-18T06:04:52.517Z'},"{'meta': {'total_results': 1, 'min_price': {'r...","{'page': 236, 'rank': 20}",,[{'date': {'$date': '2019-10-21T00:00:00.000Z'...,0.0,expensive,NaN,1160.0,1160.0,1160.0,0.0,[],[]
1,5da822607d3a9baa211f85f4,Prada Hooded rubber-appliquéd color-block gab...,1000960,https://www.net-a-porter.com/gb/en/product/100...,prada,"{'standard': [{'order': 1, 'url': 'http://cach...","{'breadcrumbs': {'1': 'Clothing ', '2': ' Jack...",5da6d40110309200045874e6,"{'offer_price': {'currency': 'GBP', 'value': 1...","Black, white and gray gabardine Concealed zip ...",netaporter_gb_pv,{'available': True},"{'l1': 'clothing', 'l2': 'outerwear', 'l3': 'c...",{'$date': '2019-10-21T00:00:00.000Z'},{'$date': '2020-01-18T06:07:47.995Z'},"{'meta': {'total_results': 1, 'min_price': {'r...","{'page': 146, 'rank': 14}",,[{'date': {'$date': '2019-10-21T00:00:00.000Z'...,100.0,cheaper,NaN,1210.0,1210.0,1210.0,0.0,[5da94e940ffeca000172b12a],[605.0]
2,5da8221a6504fb65da005a7b,Prada Printed silk crepe de chine flared pant...,1000995,https://www.net-a-porter.com/gb/en/product/100...,prada,"{'standard': [{'order': 1, 'url': 'http://cach...","{'breadcrumbs': {'1': 'Clothing ', '2': ' Pant...",5da6d40110309200045874e6,"{'offer_price': {'currency': 'GBP', 'value': 8...","Black, red and cream silk crepe de chine Conce...",netaporter_gb_pv,{'available': True},"{'l1': 'clothing', 'l2': 'bottomwear', 'l3': '...",{'$date': '2019-10-21T00:00:00.000Z'},{'$date': '2020-01-18T06:07:49.170Z'},"{'meta': {'total_results': 1, 'min_price': {'r...","{'page': 133, 'rank': 35}",,[{'date': {'$date': '2019-10-21T00:00:00.000Z'...,100.0,cheaper,NaN,855.0,855.0,855.0,0.0,[5d0cc7b68a66a100014acdb0],[518.0]
3,5da8218b6504fb65da004fb2,Prada Silk-twill shorts,1000996,https://www.net-a-porter.com/gb/en/product/100...,prada,"{'standard': [{'order': 1, 'url': 'http://cach...","{'breadcrumbs': {'1': 'Clothing ', '2': ' Shor...",5da6d40110309200045874e6,"{'offer_price': {'currency': 'GBP', 'value': 7...",Pastel-pink and black silk-twill Button and co...,netaporter_gb_pv,{'available': True},"{'l1': 'clothing', 'l2': 'bottomwear', 'l3': '...",{'$date': '2019-10-21T00:00:00.000Z'},{'$date': '2020-01-18T06:04:54.021Z'},"{'meta': {'total_results': 1, 'min_price': {'r...","{'page': 142, 'rank': 58}",,[{'date': {'$date': '2019-10-21T00:00:00.000Z'...,100.0,cheaper,NaN,700.0,700.0,700.0,0.0,[5da94e940ffeca000172b12a],[357.0]
4,5da823b76504fb65cb261ac9,Prada Plissé-organza skirt,1000997,https://www.net-a-porter.com/gb/en/product/100...,prada,"{'standard': [{'order': 1, 'url': 'http://cach...","{'breadcrumbs': {'1': 'Clothing ', '2': ' Skir...",5da6d40110309200045874e6,"{'offer_price': {'currency': 'GBP', 'value': 8...",Pastel-pink plissé-organza Concealed zip faste...,netaporter_gb_pv,{'available': True},"{'l1': 'clothing', 'l2': 'skirts', 'l3': '', '...",{'$date': '2019-10-21T00:00:00.000Z'},{'$date': '2020-01-18T06:07:51.463Z'},"{'meta': {'total_results': 1, 'min_price': {'r...","{'page': 37, 'rank': 39}",,[{'date': {'$date': '2019-10-21T00:00:00.000Z'...,100.0,cheaper,NaN,880.0,880.0,880.0,0.0,[5da94e940ffeca000172b12a],[440.0]


## Question 4
### NAP products where they are selling at a price n% higher than a competitor X.

In [0]:
# higher_list = []
# for item, basket in zip(df.similar_products, price_df.basket_price):
#   # print(item, basket)
#   temp_list = []
#   # print(item['meta'])
#   for val in item['website_results'].keys():
#     if item['website_results'][val]['meta']['total_results'] == 1:
#       # print('Yes')
#       if (basket+(basket/10)) > item['website_results'][val]['meta']['min_price']['basket']:
#         temp_list.append(val)
#     # print(item['website_results'][val]['meta']['max_price'])
#   higher_list.append(temp_list)

In [0]:
df.expensive_id = df.expensive_id.apply(lambda x: [0] if len(x)==0 else x)
df.comp_basket_price = df.comp_basket_price.apply(lambda x : [0] if len(x)==0 else x)
df.head()

,_id,name,sku,url,brand,media,meta,website_id,price,description_text,spider,stock,classification,created_at,updated_at,similar_products,positioning,lv_url,price_changes,price_positioning,price_positioning_text,sizes,offer_price,regular_price,basket_price,discount_price,expensive_id,comp_basket_price
0,5da822166504fb65da005a59,Prada Feather-embellished silk-twill robe,1000954,https://www.net-a-porter.com/gb/en/product/100...,prada,"{'standard': [{'order': 1, 'url': 'http://cach...","{'breadcrumbs': {'1': 'Clothing ', '2': ' Jack...",5da6d40110309200045874e6,"{'offer_price': {'currency': 'GBP', 'value': 1...","Gray and black silk-twill, black feathers Slip...",netaporter_gb_pv,{'available': False},"{'l1': 'clothing', 'l2': 'nightwear', 'l3': ''...",{'$date': '2019-10-21T00:00:00.000Z'},{'$date': '2020-01-18T06:04:52.517Z'},"{'meta': {'total_results': 1, 'min_price': {'r...","{'page': 236, 'rank': 20}",,[{'date': {'$date': '2019-10-21T00:00:00.000Z'...,0.0,expensive,NaN,1160.0,1160.0,1160.0,0.0,[0],[0]
1,5da822607d3a9baa211f85f4,Prada Hooded rubber-appliquéd color-block gab...,1000960,https://www.net-a-porter.com/gb/en/product/100...,prada,"{'standard': [{'order': 1, 'url': 'http://cach...","{'breadcrumbs': {'1': 'Clothing ', '2': ' Jack...",5da6d40110309200045874e6,"{'offer_price': {'currency': 'GBP', 'value': 1...","Black, white and gray gabardine Concealed zip ...",netaporter_gb_pv,{'available': True},"{'l1': 'clothing', 'l2': 'outerwear', 'l3': 'c...",{'$date': '2019-10-21T00:00:00.000Z'},{'$date': '2020-01-18T06:07:47.995Z'},"{'meta': {'total_results': 1, 'min_price': {'r...","{'page': 146, 'rank': 14}",,[{'date': {'$date': '2019-10-21T00:00:00.000Z'...,100.0,cheaper,NaN,1210.0,1210.0,1210.0,0.0,[5da94e940ffeca000172b12a],[605.0]
2,5da8221a6504fb65da005a7b,Prada Printed silk crepe de chine flared pant...,1000995,https://www.net-a-porter.com/gb/en/product/100...,prada,"{'standard': [{'order': 1, 'url': 'http://cach...","{'breadcrumbs': {'1': 'Clothing ', '2': ' Pant...",5da6d40110309200045874e6,"{'offer_price': {'currency': 'GBP', 'value': 8...","Black, red and cream silk crepe de chine Conce...",netaporter_gb_pv,{'available': True},"{'l1': 'clothing', 'l2': 'bottomwear', 'l3': '...",{'$date': '2019-10-21T00:00:00.000Z'},{'$date': '2020-01-18T06:07:49.170Z'},"{'meta': {'total_results': 1, 'min_price': {'r...","{'page': 133, 'rank': 35}",,[{'date': {'$date': '2019-10-21T00:00:00.000Z'...,100.0,cheaper,NaN,855.0,855.0,855.0,0.0,[5d0cc7b68a66a100014acdb0],[518.0]
3,5da8218b6504fb65da004fb2,Prada Silk-twill shorts,1000996,https://www.net-a-porter.com/gb/en/product/100...,prada,"{'standard': [{'order': 1, 'url': 'http://cach...","{'breadcrumbs': {'1': 'Clothing ', '2': ' Shor...",5da6d40110309200045874e6,"{'offer_price': {'currency': 'GBP', 'value': 7...",Pastel-pink and black silk-twill Button and co...,netaporter_gb_pv,{'available': True},"{'l1': 'clothing', 'l2': 'bottomwear', 'l3': '...",{'$date': '2019-10-21T00:00:00.000Z'},{'$date': '2020-01-18T06:04:54.021Z'},"{'meta': {'total_results': 1, 'min_price': {'r...","{'page': 142, 'rank': 58}",,[{'date': {'$date': '2019-10-21T00:00:00.000Z'...,100.0,cheaper,NaN,700.0,700.0,700.0,0.0,[5da94e940ffeca000172b12a],[357.0]
4,5da823b76504fb65cb261ac9,Prada Plissé-organza skirt,1000997,https://www.net-a-porter.com/gb/en/product/100...,prada,"{'standard': [{'order': 1, 'url': 'http://cach...","{'breadcrumbs': {'1': 'Clothing ', '2': ' Skir...",5da6d40110309200045874e6,"{'offer_price': {'currency': 'GBP', 'value': 8...",Pastel-pink plissé-organza Concealed zip faste...,netaporter_gb_pv,{'available': True},"{'l1': 'clothing', 'l2': 'skirts', 'l3': '', '...",{'$date': '2019-10-21T00:00:00.000Z'},{'$date': '2020-01-18T06:07:51.463Z'},"{'meta': {'total_results': 1, 'min_price': {'r...","{'page': 37, 'rank': 39}",,[{'date': {'$date': '2019-10-21T00:00:00.000Z'...,100.0,cheaper,NaN,880.0,880.0,880.0,0.0,[5da94e940ffeca000172b12a],[440.0]


In [0]:
# price_df.assign(cids=price_df['expensive_id']).explode('5d').reset_index(drop=True)
price_df2 = pd.DataFrame(df['expensive_id'].tolist(), index=df.index).stack()
comp_basket_price = pd.DataFrame(df['comp_basket_price'].tolist(), index=df.index).stack()
print(price_df2.head())
print(comp_basket_price.head())

0  0                           0
1  0    5da94e940ffeca000172b12a
2  0    5d0cc7b68a66a100014acdb0
3  0    5da94e940ffeca000172b12a
4  0    5da94e940ffeca000172b12a
dtype: object
0  0      0.0
1  0    605.0
2  0    518.0
3  0    357.0
4  0    440.0
dtype: float64


In [0]:
price_df2.head(101)

0   0                           0
1   0    5da94e940ffeca000172b12a
2   0    5d0cc7b68a66a100014acdb0
3   0    5da94e940ffeca000172b12a
4   0    5da94e940ffeca000172b12a
                   ...           
96  0    5da94e940ffeca000172b12a
97  0                           0
98  0                           0
99  0    5da94f4e6d97010001f81d72
    1    5d0cc7b68a66a100014acdb0
Length: 101, dtype: object

In [0]:
len(price_df2)

5109

In [0]:
price_df2.index = price_df2.index.droplevel(1)
comp_basket_price.index = comp_basket_price.index.droplevel(1)
print(price_df2.head())
print(comp_basket_price.head())

0                           0
1    5da94e940ffeca000172b12a
2    5d0cc7b68a66a100014acdb0
3    5da94e940ffeca000172b12a
4    5da94e940ffeca000172b12a
dtype: object
0      0.0
1    605.0
2    518.0
3    357.0
4    440.0
dtype: float64


In [0]:
df = pd.concat([df, price_df2, comp_basket_price], axis=1)
df.head()

,_id,name,sku,url,brand,media,meta,website_id,price,description_text,spider,stock,classification,created_at,updated_at,similar_products,positioning,lv_url,price_changes,price_positioning,price_positioning_text,sizes,offer_price,regular_price,basket_price,discount_price,expensive_id,comp_basket_price,0,1
0,5da822166504fb65da005a59,Prada Feather-embellished silk-twill robe,1000954,https://www.net-a-porter.com/gb/en/product/100...,prada,"{'standard': [{'order': 1, 'url': 'http://cach...","{'breadcrumbs': {'1': 'Clothing ', '2': ' Jack...",5da6d40110309200045874e6,"{'offer_price': {'currency': 'GBP', 'value': 1...","Gray and black silk-twill, black feathers Slip...",netaporter_gb_pv,{'available': False},"{'l1': 'clothing', 'l2': 'nightwear', 'l3': ''...",{'$date': '2019-10-21T00:00:00.000Z'},{'$date': '2020-01-18T06:04:52.517Z'},"{'meta': {'total_results': 1, 'min_price': {'r...","{'page': 236, 'rank': 20}",,[{'date': {'$date': '2019-10-21T00:00:00.000Z'...,0.0,expensive,NaN,1160.0,1160.0,1160.0,0.0,[0],[0],0,0.0
1,5da822607d3a9baa211f85f4,Prada Hooded rubber-appliquéd color-block gab...,1000960,https://www.net-a-porter.com/gb/en/product/100...,prada,"{'standard': [{'order': 1, 'url': 'http://cach...","{'breadcrumbs': {'1': 'Clothing ', '2': ' Jack...",5da6d40110309200045874e6,"{'offer_price': {'currency': 'GBP', 'value': 1...","Black, white and gray gabardine Concealed zip ...",netaporter_gb_pv,{'available': True},"{'l1': 'clothing', 'l2': 'outerwear', 'l3': 'c...",{'$date': '2019-10-21T00:00:00.000Z'},{'$date': '2020-01-18T06:07:47.995Z'},"{'meta': {'total_results': 1, 'min_price': {'r...","{'page': 146, 'rank': 14}",,[{'date': {'$date': '2019-10-21T00:00:00.000Z'...,100.0,cheaper,NaN,1210.0,1210.0,1210.0,0.0,[5da94e940ffeca000172b12a],[605.0],5da94e940ffeca000172b12a,605.0
2,5da8221a6504fb65da005a7b,Prada Printed silk crepe de chine flared pant...,1000995,https://www.net-a-porter.com/gb/en/product/100...,prada,"{'standard': [{'order': 1, 'url': 'http://cach...","{'breadcrumbs': {'1': 'Clothing ', '2': ' Pant...",5da6d40110309200045874e6,"{'offer_price': {'currency': 'GBP', 'value': 8...","Black, red and cream silk crepe de chine Conce...",netaporter_gb_pv,{'available': True},"{'l1': 'clothing', 'l2': 'bottomwear', 'l3': '...",{'$date': '2019-10-21T00:00:00.000Z'},{'$date': '2020-01-18T06:07:49.170Z'},"{'meta': {'total_results': 1, 'min_price': {'r...","{'page': 133, 'rank': 35}",,[{'date': {'$date': '2019-10-21T00:00:00.000Z'...,100.0,cheaper,NaN,855.0,855.0,855.0,0.0,[5d0cc7b68a66a100014acdb0],[518.0],5d0cc7b68a66a100014acdb0,518.0
3,5da8218b6504fb65da004fb2,Prada Silk-twill shorts,1000996,https://www.net-a-porter.com/gb/en/product/100...,prada,"{'standard': [{'order': 1, 'url': 'http://cach...","{'breadcrumbs': {'1': 'Clothing ', '2': ' Shor...",5da6d40110309200045874e6,"{'offer_price': {'currency': 'GBP', 'value': 7...",Pastel-pink and black silk-twill Button and co...,netaporter_gb_pv,{'available': True},"{'l1': 'clothing', 'l2': 'bottomwear', 'l3': '...",{'$date': '2019-10-21T00:00:00.000Z'},{'$date': '2020-01-18T06:04:54.021Z'},"{'meta': {'total_results': 1, 'min_price': {'r...","{'page': 142, 'rank': 58}",,[{'date': {'$date': '2019-10-21T00:00:00.000Z'...,100.0,cheaper,NaN,700.0,700.0,700.0,0.0,[5da94e940ffeca000172b12a],[357.0],5da94e940ffeca000172b12a,357.0
4,5da823b76504fb65cb261ac9,Prada Plissé-organza skirt,1000997,https://www.net-a-porter.com/gb/en/product/100...,prada,"{'standard': [{'order': 1, 'url': 'http://cach...","{'breadcrumbs': {'1': 'Clothing ', '2': ' Skir...",5da6d40110309200045874e6,"{'offer_price': {'currency': 'GBP', 'value': 8...",Pastel-pink plissé-organza Concealed zip faste...,netaporter_gb_pv,{'available': True},"{'l1': 'clothing', 'l2': 'skirts', 'l3': '', '...",{'$date': '2019-10-21T00:00:00.000Z'},{'$date': '2020-01-18T06:07:51.463Z'},"{'meta': {'total_results': 1, 'min_price': {'r...","{'page': 37, 'rank': 39}",,[{'date': {'$date': '2019-10-21T00:00:00.000Z'...,100.0,cheaper,NaN,880.0,880.0,

In [0]:
df.reset_index(inplace=True)
df.drop('index', inplace=True, axis=1)
df.head()

,_id,name,sku,url,brand,media,meta,website_id,price,description_text,spider,stock,classification,created_at,updated_at,similar_products,positioning,lv_url,price_changes,price_positioning,price_positioning_text,sizes,offer_price,regular_price,basket_price,discount_price,expensive_id,comp_basket_price,0,1
0,5da822166504fb65da005a59,Prada Feather-embellished silk-twill robe,1000954,https://www.net-a-porter.com/gb/en/product/100...,prada,"{'standard': [{'order': 1, 'url': 'http://cach...","{'breadcrumbs': {'1': 'Clothing ', '2': ' Jack...",5da6d40110309200045874e6,"{'offer_price': {'currency': 'GBP', 'value': 1...","Gray and black silk-twill, black feathers Slip...",netaporter_gb_pv,{'available': False},"{'l1': 'clothing', 'l2': 'nightwear', 'l3': ''...",{'$date': '2019-10-21T00:00:00.000Z'},{'$date': '2020-01-18T06:04:52.517Z'},"{'meta': {'total_results': 1, 'min_price': {'r...","{'page': 236, 'rank': 20}",,[{'date': {'$date': '2019-10-21T00:00:00.000Z'...,0.0,expensive,NaN,1160.0,1160.0,1160.0,0.0,[0],[0],0,0.0
1,5da822607d3a9baa211f85f4,Prada Hooded rubber-appliquéd color-block gab...,1000960,https://www.net-a-porter.com/gb/en/product/100...,prada,"{'standard': [{'order': 1, 'url': 'http://cach...","{'breadcrumbs': {'1': 'Clothing ', '2': ' Jack...",5da6d40110309200045874e6,"{'offer_price': {'currency': 'GBP', 'value': 1...","Black, white and gray gabardine Concealed zip ...",netaporter_gb_pv,{'available': True},"{'l1': 'clothing', 'l2': 'outerwear', 'l3': 'c...",{'$date': '2019-10-21T00:00:00.000Z'},{'$date': '2020-01-18T06:07:47.995Z'},"{'meta': {'total_results': 1, 'min_price': {'r...","{'page': 146, 'rank': 14}",,[{'date': {'$date': '2019-10-21T00:00:00.000Z'...,100.0,cheaper,NaN,1210.0,1210.0,1210.0,0.0,[5da94e940ffeca000172b12a],[605.0],5da94e940ffeca000172b12a,605.0
2,5da8221a6504fb65da005a7b,Prada Printed silk crepe de chine flared pant...,1000995,https://www.net-a-porter.com/gb/en/product/100...,prada,"{'standard': [{'order': 1, 'url': 'http://cach...","{'breadcrumbs': {'1': 'Clothing ', '2': ' Pant...",5da6d40110309200045874e6,"{'offer_price': {'currency': 'GBP', 'value': 8...","Black, red and cream silk crepe de chine Conce...",netaporter_gb_pv,{'available': True},"{'l1': 'clothing', 'l2': 'bottomwear', 'l3': '...",{'$date': '2019-10-21T00:00:00.000Z'},{'$date': '2020-01-18T06:07:49.170Z'},"{'meta': {'total_results': 1, 'min_price': {'r...","{'page': 133, 'rank': 35}",,[{'date': {'$date': '2019-10-21T00:00:00.000Z'...,100.0,cheaper,NaN,855.0,855.0,855.0,0.0,[5d0cc7b68a66a100014acdb0],[518.0],5d0cc7b68a66a100014acdb0,518.0
3,5da8218b6504fb65da004fb2,Prada Silk-twill shorts,1000996,https://www.net-a-porter.com/gb/en/product/100...,prada,"{'standard': [{'order': 1, 'url': 'http://cach...","{'breadcrumbs': {'1': 'Clothing ', '2': ' Shor...",5da6d40110309200045874e6,"{'offer_price': {'currency': 'GBP', 'value': 7...",Pastel-pink and black silk-twill Button and co...,netaporter_gb_pv,{'available': True},"{'l1': 'clothing', 'l2': 'bottomwear', 'l3': '...",{'$date': '2019-10-21T00:00:00.000Z'},{'$date': '2020-01-18T06:04:54.021Z'},"{'meta': {'total_results': 1, 'min_price': {'r...","{'page': 142, 'rank': 58}",,[{'date': {'$date': '2019-10-21T00:00:00.000Z'...,100.0,cheaper,NaN,700.0,700.0,700.0,0.0,[5da94e940ffeca000172b12a],[357.0],5da94e940ffeca000172b12a,357.0
4,5da823b76504fb65cb261ac9,Prada Plissé-organza skirt,1000997,https://www.net-a-porter.com/gb/en/product/100...,prada,"{'standard': [{'order': 1, 'url': 'http://cach...","{'breadcrumbs': {'1': 'Clothing ', '2': ' Skir...",5da6d40110309200045874e6,"{'offer_price': {'currency': 'GBP', 'value': 8...",Pastel-pink plissé-organza Concealed zip faste...,netaporter_gb_pv,{'available': True},"{'l1': 'clothing', 'l2': 'skirts', 'l3': '', '...",{'$date': '2019-10-21T00:00:00.000Z'},{'$date': '2020-01-18T06:07:51.463Z'},"{'meta': {'total_results': 1, 'min_price': {'r...","{'page': 37, 'rank': 39}",,[{'date': {'$date': '2019-10-21T00:00:00.000Z'...,100.0,cheaper,NaN,880.0,880.0,

In [0]:
df.rename(columns={0:'comp_website_id', 1:'basket_price_comp'}, inplace=True)
df.head()

,_id,name,sku,url,brand,media,meta,website_id,price,description_text,spider,stock,classification,created_at,updated_at,similar_products,positioning,lv_url,price_changes,price_positioning,price_positioning_text,sizes,offer_price,regular_price,basket_price,discount_price,expensive_id,comp_basket_price,comp_website_id,basket_price_comp
0,5da822166504fb65da005a59,Prada Feather-embellished silk-twill robe,1000954,https://www.net-a-porter.com/gb/en/product/100...,prada,"{'standard': [{'order': 1, 'url': 'http://cach...","{'breadcrumbs': {'1': 'Clothing ', '2': ' Jack...",5da6d40110309200045874e6,"{'offer_price': {'currency': 'GBP', 'value': 1...","Gray and black silk-twill, black feathers Slip...",netaporter_gb_pv,{'available': False},"{'l1': 'clothing', 'l2': 'nightwear', 'l3': ''...",{'$date': '2019-10-21T00:00:00.000Z'},{'$date': '2020-01-18T06:04:52.517Z'},"{'meta': {'total_results': 1, 'min_price': {'r...","{'page': 236, 'rank': 20}",,[{'date': {'$date': '2019-10-21T00:00:00.000Z'...,0.0,expensive,NaN,1160.0,1160.0,1160.0,0.0,[0],[0],0,0.0
1,5da822607d3a9baa211f85f4,Prada Hooded rubber-appliquéd color-block gab...,1000960,https://www.net-a-porter.com/gb/en/product/100...,prada,"{'standard': [{'order': 1, 'url': 'http://cach...","{'breadcrumbs': {'1': 'Clothing ', '2': ' Jack...",5da6d40110309200045874e6,"{'offer_price': {'currency': 'GBP', 'value': 1...","Black, white and gray gabardine Concealed zip ...",netaporter_gb_pv,{'available': True},"{'l1': 'clothing', 'l2': 'outerwear', 'l3': 'c...",{'$date': '2019-10-21T00:00:00.000Z'},{'$date': '2020-01-18T06:07:47.995Z'},"{'meta': {'total_results': 1, 'min_price': {'r...","{'page': 146, 'rank': 14}",,[{'date': {'$date': '2019-10-21T00:00:00.000Z'...,100.0,cheaper,NaN,1210.0,1210.0,1210.0,0.0,[5da94e940ffeca000172b12a],[605.0],5da94e940ffeca000172b12a,605.0
2,5da8221a6504fb65da005a7b,Prada Printed silk crepe de chine flared pant...,1000995,https://www.net-a-porter.com/gb/en/product/100...,prada,"{'standard': [{'order': 1, 'url': 'http://cach...","{'breadcrumbs': {'1': 'Clothing ', '2': ' Pant...",5da6d40110309200045874e6,"{'offer_price': {'currency': 'GBP', 'value': 8...","Black, red and cream silk crepe de chine Conce...",netaporter_gb_pv,{'available': True},"{'l1': 'clothing', 'l2': 'bottomwear', 'l3': '...",{'$date': '2019-10-21T00:00:00.000Z'},{'$date': '2020-01-18T06:07:49.170Z'},"{'meta': {'total_results': 1, 'min_price': {'r...","{'page': 133, 'rank': 35}",,[{'date': {'$date': '2019-10-21T00:00:00.000Z'...,100.0,cheaper,NaN,855.0,855.0,855.0,0.0,[5d0cc7b68a66a100014acdb0],[518.0],5d0cc7b68a66a100014acdb0,518.0
3,5da8218b6504fb65da004fb2,Prada Silk-twill shorts,1000996,https://www.net-a-porter.com/gb/en/product/100...,prada,"{'standard': [{'order': 1, 'url': 'http://cach...","{'breadcrumbs': {'1': 'Clothing ', '2': ' Shor...",5da6d40110309200045874e6,"{'offer_price': {'currency': 'GBP', 'value': 7...",Pastel-pink and black silk-twill Button and co...,netaporter_gb_pv,{'available': True},"{'l1': 'clothing', 'l2': 'bottomwear', 'l3': '...",{'$date': '2019-10-21T00:00:00.000Z'},{'$date': '2020-01-18T06:04:54.021Z'},"{'meta': {'total_results': 1, 'min_price': {'r...","{'page': 142, 'rank': 58}",,[{'date': {'$date': '2019-10-21T00:00:00.000Z'...,100.0,cheaper,NaN,700.0,700.0,700.0,0.0,[5da94e940ffeca000172b12a],[357.0],5da94e940ffeca000172b12a,357.0
4,5da823b76504fb65cb261ac9,Prada Plissé-organza skirt,1000997,https://www.net-a-porter.com/gb/en/product/100...,prada,"{'standard': [{'order': 1, 'url': 'http://cach...","{'breadcrumbs': {'1': 'Clothing ', '2': ' Skir...",5da6d40110309200045874e6,"{'offer_price': {'currency': 'GBP', 'value': 8...",Pastel-pink plissé-organza Concealed zip faste...,netaporter_gb_pv,{'available': True},"{'l1': 'clothing', 'l2': 'skirts', 'l3': '', '...",{'$date': '2019-10-21T00:00:00.000Z'},{'$date': '2020-01-18T06:07:51.463Z'},"{'meta': {'total_results': 1, 'min_price': {'r...","{'page': 37, 'rank': 39}",,[{'date': {'$date': '2019-10-21T00:00:00.000Z'...,

In [0]:
# price_data['basket_fluctuation_price'] = [(price_data['basket_price'] - price_data['basket_price_comp'])/price_data['basket_price_comp'] if price_data['basket_price_comp']!=0 else 0]
# price_data.head()
df['basket_fluctuation_price'] = np.where(df['basket_price_comp'] != 0, (df['basket_price']-df['basket_price_comp'])/df['basket_price_comp'], 0)
df.head()

,_id,name,sku,url,brand,media,meta,website_id,price,description_text,spider,stock,classification,created_at,updated_at,similar_products,positioning,lv_url,price_changes,price_positioning,price_positioning_text,sizes,offer_price,regular_price,basket_price,discount_price,expensive_id,comp_basket_price,comp_website_id,basket_price_comp,basket_fluctuation_price
0,5da822166504fb65da005a59,Prada Feather-embellished silk-twill robe,1000954,https://www.net-a-porter.com/gb/en/product/100...,prada,"{'standard': [{'order': 1, 'url': 'http://cach...","{'breadcrumbs': {'1': 'Clothing ', '2': ' Jack...",5da6d40110309200045874e6,"{'offer_price': {'currency': 'GBP', 'value': 1...","Gray and black silk-twill, black feathers Slip...",netaporter_gb_pv,{'available': False},"{'l1': 'clothing', 'l2': 'nightwear', 'l3': ''...",{'$date': '2019-10-21T00:00:00.000Z'},{'$date': '2020-01-18T06:04:52.517Z'},"{'meta': {'total_results': 1, 'min_price': {'r...","{'page': 236, 'rank': 20}",,[{'date': {'$date': '2019-10-21T00:00:00.000Z'...,0.0,expensive,NaN,1160.0,1160.0,1160.0,0.0,[0],[0],0,0.0,0.000000
1,5da822607d3a9baa211f85f4,Prada Hooded rubber-appliquéd color-block gab...,1000960,https://www.net-a-porter.com/gb/en/product/100...,prada,"{'standard': [{'order': 1, 'url': 'http://cach...","{'breadcrumbs': {'1': 'Clothing ', '2': ' Jack...",5da6d40110309200045874e6,"{'offer_price': {'currency': 'GBP', 'value': 1...","Black, white and gray gabardine Concealed zip ...",netaporter_gb_pv,{'available': True},"{'l1': 'clothing', 'l2': 'outerwear', 'l3': 'c...",{'$date': '2019-10-21T00:00:00.000Z'},{'$date': '2020-01-18T06:07:47.995Z'},"{'meta': {'total_results': 1, 'min_price': {'r...","{'page': 146, 'rank': 14}",,[{'date': {'$date': '2019-10-21T00:00:00.000Z'...,100.0,cheaper,NaN,1210.0,1210.0,1210.0,0.0,[5da94e940ffeca000172b12a],[605.0],5da94e940ffeca000172b12a,605.0,1.000000
2,5da8221a6504fb65da005a7b,Prada Printed silk crepe de chine flared pant...,1000995,https://www.net-a-porter.com/gb/en/product/100...,prada,"{'standard': [{'order': 1, 'url': 'http://cach...","{'breadcrumbs': {'1': 'Clothing ', '2': ' Pant...",5da6d40110309200045874e6,"{'offer_price': {'currency': 'GBP', 'value': 8...","Black, red and cream silk crepe de chine Conce...",netaporter_gb_pv,{'available': True},"{'l1': 'clothing', 'l2': 'bottomwear', 'l3': '...",{'$date': '2019-10-21T00:00:00.000Z'},{'$date': '2020-01-18T06:07:49.170Z'},"{'meta': {'total_results': 1, 'min_price': {'r...","{'page': 133, 'rank': 35}",,[{'date': {'$date': '2019-10-21T00:00:00.000Z'...,100.0,cheaper,NaN,855.0,855.0,855.0,0.0,[5d0cc7b68a66a100014acdb0],[518.0],5d0cc7b68a66a100014acdb0,518.0,0.650579
3,5da8218b6504fb65da004fb2,Prada Silk-twill shorts,1000996,https://www.net-a-porter.com/gb/en/product/100...,prada,"{'standard': [{'order': 1, 'url': 'http://cach...","{'breadcrumbs': {'1': 'Clothing ', '2': ' Shor...",5da6d40110309200045874e6,"{'offer_price': {'currency': 'GBP', 'value': 7...",Pastel-pink and black silk-twill Button and co...,netaporter_gb_pv,{'available': True},"{'l1': 'clothing', 'l2': 'bottomwear', 'l3': '...",{'$date': '2019-10-21T00:00:00.000Z'},{'$date': '2020-01-18T06:04:54.021Z'},"{'meta': {'total_results': 1, 'min_price': {'r...","{'page': 142, 'rank': 58}",,[{'date': {'$date': '2019-10-21T00:00:00.000Z'...,100.0,cheaper,NaN,700.0,700.0,700.0,0.0,[5da94e940ffeca000172b12a],[357.0],5da94e940ffeca000172b12a,357.0,0.960784
4,5da823b76504fb65cb261ac9,Prada Plissé-organza skirt,1000997,https://www.net-a-porter.com/gb/en/product/100...,prada,"{'standard': [{'order': 1, 'url': 'http://cach...","{'breadcrumbs': {'1': 'Clothing ', '2': ' Skir...",5da6d40110309200045874e6,"{'offer_price': {'currency': 'GBP', 'value': 8...",Pastel-pink plissé-organza Concealed zip faste...,netaporter_gb_pv,{'available': True},"{'l1': 'clothing', 'l2': 'skirts', 'l3': '', '...",{'$date': '2019-10-21T00:00:00.000Z'},{'$date': '2020-01-18T06:07:51.463Z'},"{'meta': {'total_results': 1, 'min_price': {'r...","{'page': 37, 'ra

In [0]:
# price_data['basket_fluctuation_price'] = price_data.apply(lambda x : (x['basket_price']-x['basket_price_comp'])/x['basket_price_comp'] if x['basket_price_comp'] != 0 else 0)
# price_data.head()

In [0]:
df['basket_fluctuation_price'] = df['basket_fluctuation_price']*100

In [0]:
demo = df.loc[df['discount_price']>240]
demo.head()

,_id,name,sku,url,brand,media,meta,website_id,price,description_text,spider,stock,classification,created_at,updated_at,similar_products,positioning,lv_url,price_changes,price_positioning,price_positioning_text,sizes,offer_price,regular_price,basket_price,discount_price,expensive_id,comp_basket_price,comp_website_id,basket_price_comp,basket_fluctuation_price
171,5da821d3b25b035d1d17bbea,JW Anderson Disc leather and suede shoulder b...,1065872,https://www.net-a-porter.com/gb/en/product/106...,jw anderson,"{'standard': [{'order': 1, 'url': 'http://cach...","{'breadcrumbs': {'1': 'Bags ', '2': ' Shoulder...",5da6d40110309200045874e6,"{'offer_price': {'currency': 'GBP', 'value': 5...",Black suede and leather (Cow) Snap-fastening f...,netaporter_gb_pv,{'available': True},"{'l1': 'bags and wallets and purses', 'l2': 'b...",{'$date': '2019-10-21T00:00:00.000Z'},{'$date': '2020-01-18T06:08:06.016Z'},"{'meta': {'total_results': 1, 'min_price': {'r...","{'page': 0, 'rank': 8}",,[{'date': {'$date': '2019-10-21T00:00:00.000Z'...,0.000000,expensive,NaN,540.0,1350.0,540.0,810.0,[0],[0],0,0.0,0.000000
178,5da821b4b25b035d1d17ba13,SAINT LAURENT Satin-trimmed velvet blazer,1068852,https://www.net-a-porter.com/gb/en/product/106...,saint laurent,"{'standard': [{'order': 1, 'url': 'http://cach...","{'breadcrumbs': {'1': 'Clothing ', '2': ' Jack...",5da6d40110309200045874e6,"{'offer_price': {'currency': 'GBP', 'value': 1...",Black velvet and satin Button fastening at fro...,netaporter_gb_pv,{'available': True},"{'l1': 'clothing', 'l2': 'outerwear', 'l3': 'b...",{'$date': '2019-10-22T00:00:00.000Z'},{'$date': '2020-01-18T06:05:06.815Z'},"{'meta': {'total_results': 3, 'min_price': {'r...","{'page': 0, 'rank': 35}",,[{'date': {'$date': '2019-10-22T00:00:00.000Z'...,65.895062,cheaper,NaN,1296.0,2160.0,1296.0,864.0,"[5da94ef80ffeca000172b12c, 5da94e940ffeca00017...","[869.0, 1293.0]",5da94ef80ffeca000172b12c,869.0,0.491369
179,5da821b4b25b035d1d17ba13,SAINT LAURENT Satin-trimmed velvet blazer,1068852,https://www.net-a-porter.com/gb/en/product/106...,saint laurent,"{'standard': [{'order': 1, 'url': 'http://cach...","{'breadcrumbs': {'1': 'Clothing ', '2': ' Jack...",5da6d40110309200045874e6,"{'offer_price': {'currency': 'GBP', 'value': 1...",Black velvet and satin Button fastening at fro...,netaporter_gb_pv,{'available': True},"{'l1': 'clothing', 'l2': 'outerwear', 'l3': 'b...",{'$date': '2019-10-22T00:00:00.000Z'},{'$date': '2020-01-18T06:05:06.815Z'},"{'meta': {'total_results': 3, 'min_price': {'r...","{'page': 0, 'rank': 35}",,[{'date': {'$date': '2019-10-22T00:00:00.000Z'...,65.895062,cheaper,NaN,1296.0,2160.0,1296.0,864.0,"[5da94ef80ffeca000172b12c, 5da94e940ffeca00017...","[869.0, 1293.0]",5da94e940ffeca000172b12a,1293.0,0.002320
186,5da8217db25b035d1d17b6b3,Alexander McQueen Layered grosgrain-trimmed d...,1069854,https://www.net-a-porter.com/gb/en/product/106...,alexander mcqueen,"{'standard': [{'order': 1, 'url': 'http://cach...","{'breadcrumbs': {'1': 'Clothing ', '2': ' Jack...",5da6d40110309200045874e6,"{'offer_price': {'currency': 'GBP', 'value': 7...","Black denim, black, off-white and red grosgrai...",netaporter_gb_pv,{'available': True},"{'l1': 'clothing', 'l2': 'outerwear', 'l3': 'c...",{'$date': '2019-10-22T00:00:00.000Z'},{'$date': '2020-01-18T06:05:06.404Z'},"{'meta': {'total_results': 1, 'min_price': {'r...","{'page': 260, 'rank': 20}",,[{'date': {'$date': '2019-10-22T00:00:00.000Z'...,0.000000,expensive,NaN,797.5,1595.0,797.5,797.5,[0],[0],0,0.0,0.000000
208,5da81f027861c2af6a5a92e5,Balenciaga Ville XXS AJ printed textured-leat...,1074633,https://www.net-a-porter.com/gb/en/product/107...,balenciaga,"{'standard': [{'order': 1, 'url': 'http://cach...","{'breadcrumbs': {'1': 'Bags ', '2': ' Tote Bag...",5da6d40110309200045874e6,"{'offer_price': {'currency': 'GBP', 'value': 7...",Red and black textured-leather (Calf) Zip fast...,netaporter_gb_pv,{'available': True},"{'l1': 'bags and wallets and purses', 'l2': 'p...",{'$date': '2019-10-21

In [0]:
from google.colab import files
df.to_csv('nap_retailer.csv')
files.download("nap_retailer.csv")

In [0]:
discount_by_brand.to_csv('discount_nap_retailer.csv')
files.download("discount_nap_retailer.csv")

## Flask Query

In [0]:
product = df.loc[df['discount_price'] > 10]
product_avg = product['discount_price'].mean()
print(product_avg)
product_count = len(product.index)
print(product_count)

425.89668388915
2129


In [0]:
brand_count = df.brand.value_counts()
type(brand_count)

pandas.core.series.Series

In [0]:
df.groupby('brand')['discount_price'].mean()

brand
16arlington                                    189.000000
3.1 phillip lim                                 96.055556
a.p.c. atelier de production et de création      0.000000
a.w.a.k.e. mode                                116.142857
acne studios                                   136.935714
                                                  ...    
wandler                                         84.642857
we11done                                       394.833333
wolford                                          0.000000
yuzefi                                         136.500000
zimmermann                                     189.507576
Name: discount_price, Length: 359, dtype: float64

In [0]:
df.groupby('brand')['discount_price'].mean()['16arlington']

189.0

In [0]:
type(df.loc[df['brand']=='16arlington']['discount_price'].mean())

float

In [0]:
brand_count = df['brand'].value_counts()
product_count = brand_count['16arlington']
avg_discount = df.groupby('brand')['discount_price'].mean()['16arlington']

In [0]:
json = {'count':product_count, 'avg':avg_discount}
json

{'avg': 189.0, 'count': 1}